In [32]:
import pandas as pd
import os
import copy
import unittest
import string
import re
import ast

def read_file(trainFile):
    pwd = os.getcwd()
    os.chdir(os.path.dirname(trainFile))
    file1 = open(trainFile, 'r') 
    lines = file1.read().splitlines()
    return lines

In [33]:
lines = read_file("C:/Code/advent/2021_day18_input.txt")

In [34]:
lines_example1 = read_file("C:/Code/advent/2021_day18_example1.txt")
lines_example1

['[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
 '[[[5,[2,8]],4],[5,[[9,9],0]]]',
 '[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]',
 '[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]',
 '[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]',
 '[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]',
 '[[[[5,4],[7,7]],8],[[8,3],8]]',
 '[[9,3],[[9,9],[6,[4,9]]]]',
 '[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]',
 '[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]']

In [35]:
def parse(input):        
    return list(map(ast.literal_eval,input))

parse(lines_example1)

[[[[0, [5, 8]], [[1, 7], [9, 6]]], [[4, [1, 2]], [[1, 4], 2]]],
 [[[5, [2, 8]], 4], [5, [[9, 9], 0]]],
 [6, [[[6, 2], [5, 6]], [[7, 6], [4, 7]]]],
 [[[6, [0, 7]], [0, 9]], [4, [9, [9, 0]]]],
 [[[7, [6, 4]], [3, [1, 3]]], [[[5, 5], 1], 9]],
 [[6, [[7, 3], [3, 2]]], [[[3, 8], [5, 7]], 4]],
 [[[[5, 4], [7, 7]], 8], [[8, 3], 8]],
 [[9, 3], [[9, 9], [6, [4, 9]]]],
 [[2, [[7, 7], 7]], [[5, 8], [[9, 3], [0, 2]]]],
 [[[[5, 2], 5], [8, [3, 7]]], [[5, [7, 5]], [4, 4]]]]

In [73]:
def make_flat_with_depth(input, depth):
    if type(input) is list:
        left = make_flat_with_depth(input[0], depth+1)
        right = make_flat_with_depth(input[1], depth+1)
        result = []
        for v in left:
            result.append(v)
        for v in right:
            result.append(v)
        return result
    else:
        return [(input, depth)]

def make_flat(input):
    return make_flat_with_depth(input, 1)

print(make_flat([1,2]))
print(make_flat([[1,2],[[3,4],5]]))
print(make_flat([[[[0,7],4],[[7,8],[6,0]]],[8,1]]))
print(make_flat([[[[1,1],[2,2]],[3,3]],[4,4]]))
print(make_flat([[[[3,0],[5,3]],[4,4]],[5,5]]))
print(make_flat([[[[5,0],[7,4]],[5,5]],[6,6]]))
print(make_flat([[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]))

[(1, 2), (2, 2)]
[(1, 3), (2, 3), (3, 4), (4, 4), (5, 3)]
[(0, 5), (7, 5), (4, 4), (7, 5), (8, 5), (6, 5), (0, 5), (8, 3), (1, 3)]
[(1, 5), (1, 5), (2, 5), (2, 5), (3, 4), (3, 4), (4, 3), (4, 3)]
[(3, 5), (0, 5), (5, 5), (3, 5), (4, 4), (4, 4), (5, 3), (5, 3)]
[(5, 5), (0, 5), (7, 5), (4, 5), (5, 4), (5, 4), (6, 3), (6, 3)]
[(8, 5), (7, 5), (7, 5), (7, 5), (8, 5), (6, 5), (7, 5), (7, 5), (0, 5), (7, 5), (6, 5), (6, 5), (8, 4), (7, 4)]


In [83]:
def find_pair_index(input):
    for i in range(len(input)-1):
        if input[i][1] == input[i+1][1]:
            return i
    return None

def calculate_magnitude_flat(input):
    if len(input) == 1:
        return input[0][0]
    index = find_pair_index(input)
    left = input[index]
    right = input[index+1]
    new_depth = input[index][1]-1
    subresult = (3*left[0] + 2*right[0], new_depth)
    result = []
    if index > 0:
        for v in input[0:index]:
            result.append(v)       
    result.append(subresult)
    if index < len(input)-2:
        for v in input[index+2:]:
            result.append(v)
    return calculate_magnitude_flat(result)

print(calculate_magnitude_flat(make_flat([1,2])))
print(calculate_magnitude_flat(make_flat([[1,2],[[3,4],5]])))
print(calculate_magnitude_flat(make_flat([[[[0,7],4],[[7,8],[6,0]]],[8,1]])))
print(calculate_magnitude_flat(make_flat([[[[1,1],[2,2]],[3,3]],[4,4]])))
print(calculate_magnitude_flat(make_flat([[[[3,0],[5,3]],[4,4]],[5,5]])))
print(calculate_magnitude_flat(make_flat([[[[5,0],[7,4]],[5,5]],[6,6]])))
print(calculate_magnitude_flat(make_flat([[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]])))


7
143
1384
445
791
1137
3488


In [96]:
def from_flat(input):
    if len(input) == 1:
        return input[0][0]
    index = find_pair_index(input)
    left = input[index]
    right = input[index+1]
    new_depth = input[index][1]-1
    subresult = ([left[0],right[0]], new_depth)
    result = []
    if index > 0:
        for v in input[0:index]:
            result.append(v)       
    result.append(subresult)
    if index < len(input)-2:
        for v in input[index+2:]:
            result.append(v)
    return from_flat(result)

print(from_flat(make_flat([1,2])))
print(from_flat(make_flat([[1,2],[[3,4],5]])))
print(from_flat(make_flat([[[[0,7],4],[[7,8],[6,0]]],[8,1]])))
print(from_flat(make_flat([[[[1,1],[2,2]],[3,3]],[4,4]])))
print(from_flat(make_flat([[[[3,0],[5,3]],[4,4]],[5,5]])))
print(from_flat(make_flat([[[[5,0],[7,4]],[5,5]],[6,6]])))
print(from_flat(make_flat([[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]])))

[1, 2]
[[1, 2], [[3, 4], 5]]
[[[[0, 7], 4], [[7, 8], [6, 0]]], [8, 1]]
[[[[1, 1], [2, 2]], [3, 3]], [4, 4]]
[[[[3, 0], [5, 3]], [4, 4]], [5, 5]]
[[[[5, 0], [7, 4]], [5, 5]], [6, 6]]
[[[[8, 7], [7, 7]], [[8, 6], [7, 7]]], [[[0, 7], [6, 6]], [8, 7]]]


In [106]:
def find_pair_index_with_depth(input, depth):
    for i in range(len(input)-1):
        if input[i][1] < depth:
            continue
        if input[i][1] == input[i+1][1]:
            return i
    return None

def explode_flat(input,debug):
    index = find_pair_index_with_depth(input,6)
    if index == None:
        return None
    left = input[index]
    right = input[index+1]
    new_depth = input[index][1]-1
    result = []
    if index > 0:
        for j in range(0,index):
            v = input[j]
            if j == index-1:
                v = (input[j][0] + left[0],input[j][1])
            result.append(v)       
    subresult = (0, new_depth)
    result.append(subresult)
    if index < len(input)-2:
        for j in range(index+2, len(input)):
            v = input[j]
            if j == index+2:
                v = (input[j][0] + right[0],input[j][1])
            result.append(v)
    return result

def test_explode_flat(input):
    flat = make_flat(input)
    exploded = explode_flat(flat,False)
    result = from_flat(exploded)
    print(result)


print(explode_flat(make_flat([[6,[5,[4,[3,2]]]],1]), True))
print(explode_flat(make_flat([[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]),True))
print('------------------------------')
test_explode_flat([[[[[9,8],1],2],3],4])
test_explode_flat([7,[6,[5,[4,[3,2]]]]])
test_explode_flat([[6,[5,[4,[3,2]]]],1])
test_explode_flat([[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]])
test_explode_flat([[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]])


[(6, 3), (5, 4), (7, 5), (0, 5), (3, 2)]
[(3, 3), (2, 4), (8, 5), (0, 5), (9, 3), (5, 4), (4, 5), (3, 6), (2, 6)]
------------------------------
[[[[0, 9], 2], 3], 4]
[7, [6, [5, [7, 0]]]]
[[6, [5, [7, 0]]], 3]
[[3, [2, [8, 0]]], [9, [5, [4, [3, 2]]]]]
[[3, [2, [8, 0]]], [9, [5, [7, 0]]]]


In [110]:
def find_index_with_large_value(input, min_value):
    for i in range(len(input)):
        if input[i][0] < min_value:
            continue
        return i
    return None

def split_flat(input,debug):
    index = find_index_with_large_value(input,10)
    if index == None:
        return None
    target = input[index]
    new_depth = input[index][1]+1
    result = []
    if index > 0:
        for j in range(0,index):
            v = input[j]
            result.append(v)
    left_value = target[0] // 2
    right_value = target[0] - left_value
    subresult_left = (left_value, new_depth)
    subresult_right = (right_value, new_depth)
    result.append(subresult_left)
    result.append(subresult_right)
    if index < len(input)-1:
        for j in range(index+1, len(input)):
            v = input[j]
            result.append(v)
    return result

def test_split_flat(input):
    flat = make_flat(input)
    splitted = split_flat(flat,False)
    result = from_flat(splitted)
    print(result)

test_split_flat([[[[0,7],4],[15,[0,13]]],[1,1]])
test_split_flat([[[[0, 7], 4], [[7, 8], [0, 13]]], [1, 1]])

[[[[0, 7], 4], [[7, 8], [0, 13]]], [1, 1]]
[[[[0, 7], 4], [[7, 8], [0, [6, 7]]]], [1, 1]]


In [114]:
def reduce_flat(input, debug):
    data = input
    while True:        
        exploded = explode_flat(data,False)
        if exploded != None:
            if debug:
                print('after explode', exploded)
            data = exploded
            continue
        splitted = split_flat(data,False)
        if splitted != None:
            if debug:
                print('after split', data)
            data = splitted
            continue
        break
    return data  

def test_reduce_flat(input):
    flat = make_flat(input)
    reduced = reduce_flat(flat,True)
    result = from_flat(reduced)
    print(result)

test_reduce_flat([[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]])

after explode [(0, 5), (7, 5), (4, 4), (7, 4), (8, 6), (4, 6), (9, 5), (1, 3), (1, 3)]
after explode [(0, 5), (7, 5), (4, 4), (15, 4), (0, 5), (13, 5), (1, 3), (1, 3)]
after split [(0, 5), (7, 5), (4, 4), (15, 4), (0, 5), (13, 5), (1, 3), (1, 3)]
after split [(0, 5), (7, 5), (4, 4), (7, 5), (8, 5), (0, 5), (13, 5), (1, 3), (1, 3)]
after explode [(0, 5), (7, 5), (4, 4), (7, 5), (8, 5), (6, 5), (0, 5), (8, 3), (1, 3)]
[[[[0, 7], 4], [[7, 8], [6, 0]]], [8, 1]]


In [87]:
def add_flat(first, second, debug):
    result = []
    for v in first:
        result.append((v[0],v[1]+1))
    for v in second:
        result.append((v[0],v[1]+1))        
    if debug:
        print('after addition', result)
    return reduce_flat(result, debug)

add_flat(make_flat([[[[4,3],4],4],[7,[[8,4],9]]]) , make_flat([1,1]), True)

after addition [(4, 6), (3, 6), (4, 5), (4, 4), (7, 4), (8, 6), (4, 6), (9, 5), (1, 3), (1, 3)]


[(4, 6),
 (3, 6),
 (4, 5),
 (4, 4),
 (7, 4),
 (8, 6),
 (4, 6),
 (9, 5),
 (1, 3),
 (1, 3)]

In [119]:
def calculate_part_one(input, debug):
    current = make_flat(input[0])
    for i in range(1,len(input)):
        current = add_flat(current, make_flat(input[i]), debug)
    
    return calculate_magnitude_flat(current)
    
calculate_part_one(parse(lines_example1), False)

4140

In [121]:
calculate_part_one(parse(lines), False)

4207

In [130]:
def calculate_sum_magnitude(left, right):
    first = make_flat(left)
    second = make_flat(right)
    total = add_flat(first, second, False)
    value = calculate_magnitude_flat(total)
    return value

def calculate_part_two(input):
    maximum = 0
    for i in range(0,len(input)):
        for j in range(0,len(input)):
            value = calculate_sum_magnitude(input[i], input[j])
            if value > maximum:
                maximum = value
            #print(i,j,value, maximum)                
    return maximum

calculate_part_two(parse(lines_example1))

3993

In [131]:
calculate_part_two(parse(lines))

4635